In [8]:
import numpy as np
import multiversx_utils as mu
import json
import pandas as pd
import dtale

collections = ['CRMYTH-546419', 'CRWEAPONS-e5ab49', 'GSPACEAPE-08bc2b', 'CEA-2d29f9', 'CRCHAMPS-d0265d', 'CRHEROES-9edff2']
weapons = ['CRMYTH-546419', 'CRWEAPONS-e5ab49']
champions = ['GSPACEAPE-08bc2b', 'CEA-2d29f9', 'CRCHAMPS-d0265d', 'CRHEROES-9edff2']
collections = ['CRMYTH-546419', 'CRCHAMPS-d0265d', 'CRHEROES-9edff2', 'CRWEAPONS-e5ab49']

def foo(priceAmount, starLevel):
    if priceAmount is None:
        return priceAmount
    return priceAmount / (max(1, 3*(starLevel-1)))
weapon_df = {}
champion_df = {}
for collection_name in collections:
    with open(f'{collection_name}_nfts.json') as f:
        collection_nfts = json.load(f)
    for identifier, data in collection_nfts.items():
        if collection_name in weapons:
            weapon_row = {}
            weapon_row['collection'] = collection_name
            weapon_row['identifier'] = identifier
            weapon_row['name'] = data['name']
            weapon_row['priceCurrency'] = data['offchain_data']['price']['currency']
            weapon_row['priceAmount'] = data['offchain_data']['price']['amount']
            for url in data['offchain_data']:
                if identifier.startswith('CRMYTH-546419') and url.startswith('https://metadata.cantinaroyale.io/dynamic/'):
                    weapon_row.update(mu.parse_weapon_dynamic_data(data['offchain_data'][url]))
                elif identifier.startswith('CRWEAPONS-e5ab49') and url.startswith('https://metadata.cantinaroyale.io/dynamic/'):
                    weapon_row.update(mu.parse_weapon_dynamic_data(data['offchain_data'][url]))
            weapon_row['realPriceAmount'] = foo(weapon_row['priceAmount'], weapon_row['starLevel'])
            for key, value in weapon_row.items():
                if key not in weapon_df:
                    weapon_df[key] = []
                weapon_df[key].append(value)
        elif collection_name in champions:
            champion_row = {}
            champion_row['collection'] = collection_name
            champion_row['identifier'] = identifier
            champion_row['priceCurrency'] = data['offchain_data']['price']['currency']
            champion_row['priceAmount'] = data['offchain_data']['price']['amount']
            for url in data:
                if identifier.startswith('CRHEROES-9edff2') and url.startswith('https://metadata.cantinaroyale.io/dynamic/'):
                    champion_row.update(mu.parse_champion_dynamic_data(data['offchain_data'][url]))
                if identifier.startswith('CRCHAMPS-d0265d') and url.startswith('https://metadata.cantinaroyale.io/dynamic/'):
                    champion_row.update(mu.parse_champion_dynamic_data(data['offchain_data'][url]))
                elif identifier.startswith('GSPACEAPE-08bc2b') and url.startswith('https://metadata.verko.io/dynamic/'):
                    champion_row.update(mu.parse_champion_dynamic_data(data['offchain_data'][url]))
                elif identifier.startswith('CEA-2d29f9') and url.startswith('https://metadata.verko.io/dynamic/'):
                    champion_row.update(mu.parse_champion_dynamic_data(data['offchain_data'][url]))
            for key, value in champion_row.items():
                if key not in champion_df:
                    champion_df[key] = []
                champion_df[key].append(value)
weapon_df = pd.DataFrame(weapon_df)
champion_df = pd.DataFrame(champion_df)
d = dtale.show(weapon_df)
d.open_browser()
d = dtale.show(champion_df)
d.open_browser()